# 🎬 Movie Recommendation System (Interactive)
Select your favorite movie from the dropdown to get recommendations.

In [ ]:
# Import dependencies
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Load and preprocess dataset
DATA_PATH = '/content/movies.csv'  # Replace with your CSV path
movies_data = pd.read_csv(DATA_PATH)
selected_features = ['genres','keywords','tagline','cast','director']
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')
combined_features = (movies_data['genres'] + ' ' +
                     movies_data['keywords'] + ' ' +
                     movies_data['tagline'] + ' ' +
                     movies_data['cast'] + ' ' +
                     movies_data['director'])

In [ ]:
# Convert text to vectors and compute similarity
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
similarity = cosine_similarity(feature_vectors)

In [ ]:
# Function to recommend movies
def recommend_movies(movie_name, movies_data, similarity, top_n=10):
    list_of_all_titles = movies_data['title'].tolist()
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    if not find_close_match:
        print(f"❌ No match found for '{movie_name}'")
        return []
    close_match = find_close_match[0]
    index_of_the_movie = movies_data[movies_data['title'] == close_match].index[0]
    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    print(f"\n🎥 Movies suggested for you based on '{close_match}':\n")
    for i, movie in enumerate(sorted_similar_movies[1:top_n+1], 1):
        index = movie[0]
        print(f"{i}. {movies_data.iloc[index]['title']}")
    return [movies_data.iloc[movie[0]]['title'] for movie in sorted_similar_movies[1:top_n+1]]

In [ ]:
# Create dropdown for movie selection
movie_dropdown = widgets.Dropdown(
    options=movies_data['title'].tolist(),
    description='Movie:',
    disabled=False,
    layout=widgets.Layout(width='70%')
)

recommend_button = widgets.Button(description="Get Recommendations", button_style='success')

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        recommend_movies(movie_dropdown.value, movies_data, similarity, top_n=10)

recommend_button.on_click(on_button_click)
display(movie_dropdown, recommend_button, output)